# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [3]:
pip install scipy==1.2

  Using cached scipy-1.2.0.tar.gz (23.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for scipy
Failed to build scipy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.8.0
    Uninstalling scipy-1.8.0:
      Successfully uninstalled scipy-1.8.0
  Running setup.py install for scipy: started
  Running setup.py install for scipy: finished with status 'error'Note: you may need to restart the kernel to use updated packages.
  Rolling back uninstall of scipy
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy-1.8.0.dist-info\
   from C:\Users\raman\anaconda3\Lib\site-packages\~cipy-1.8.0.dist-info


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [128 lines of output]
  C:\Users\raman\AppData\Local\Temp\pip-install-zw0jjbra\scipy_1c42564f30504338bcfb47afe14d99e6\setup.py:114: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
    import imp
  C:\Users\raman\AppData\Local\Temp\pip-install-zw0jjbra\scipy_1c42564f30504338bcfb47afe14d99e6\setup.py:468: DeprecationWarning:
  
    `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
    of the deprecation of `distutils` itself. It will be removed for
    Python >= 3.12. For older Python versions it will remain present.
    It is recommended to use `setuptools < 60.0` for those Python versions.
    For more details, see:
      https://numpy.org/devdocs/reference/distutils_status_migration.html
  
  
    from numpy.distutils.core import setup
  Running from scipy source directory.
  INFO


  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\.libs\
   from C:\Users\raman\anaconda3\Lib\site-packages\scipy\~libs
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\__config__.py
   from C:\Users\raman\AppData\Local\Temp\pip-uninstall-3uvyjifr\__config__.py
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\__init__.py
   from C:\Users\raman\AppData\Local\Temp\pip-uninstall-3uvyjifr\__init__.py
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\__pycache__\
   from C:\Users\raman\anaconda3\Lib\site-packages\scipy\~_pycache__
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\_build_utils\
   from C:\Users\raman\anaconda3\Lib\site-packages\scipy\~build_utils
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\_distributor_init.py
   from C:\Users\raman\AppData\Local\Temp\pip-uninstall-3uvyjifr\_distributor_init.py
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\_lib\
   from C:\Users\raman\anaconda3\Lib\site

  INFO:   libraries mkl_rt not found in ['C:/Users/raman/anaconda3\\Library\\lib']
  INFO:   NOT AVAILABLE
  INFO:
  INFO: openblas_lapack_info:
  INFO:   libraries openblas not found in ['C:\\Users\\raman\\anaconda3\\lib', 'C:\\', 'C:\\Users\\raman\\anaconda3\\libs']
  INFO: get_default_fcompiler: matching types: '['gnu', 'intelv', 'absoft', 'compaqv', 'intelev', 'gnu95', 'g95', 'intelvem', 'intelem', 'flang']'
  INFO: customize GnuFCompiler
  WARN: Could not locate executable g77
  WARN: Could not locate executable f77
  INFO: customize IntelVisualFCompiler
  WARN: Could not locate executable ifort
  WARN: Could not locate executable ifl
  INFO: customize AbsoftFCompiler
  WARN: Could not locate executable f90
  INFO: customize CompaqVisualFCompiler
  WARN: Could not locate executable DF
  INFO: customize IntelItaniumVisualFCompiler
  WARN: Could not locate executable efl
  INFO: customize Gnu95FCompiler
  WARN: Could not locate executable gfortran
  WARN: Could not locate executable

   from C:\Users\raman\AppData\Local\Temp\pip-uninstall-3uvyjifr\integrate\_ode.py
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\integrate\_odepack.cp38-win_amd64.pyd
   from C:\Users\raman\AppData\Local\Temp\pip-uninstall-3uvyjifr\integrate\_odepack.cp38-win_amd64.pyd
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\integrate\_quad_vec.py
   from C:\Users\raman\AppData\Local\Temp\pip-uninstall-3uvyjifr\integrate\_quad_vec.py
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\integrate\_quadpack.cp38-win_amd64.pyd
   from C:\Users\raman\AppData\Local\Temp\pip-uninstall-3uvyjifr\integrate\_quadpack.cp38-win_amd64.pyd
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\integrate\_quadrature.py
   from C:\Users\raman\AppData\Local\Temp\pip-uninstall-3uvyjifr\integrate\_quadrature.py
  Moving to c:\users\raman\anaconda3\lib\site-packages\scipy\integrate\_test_multivariate.cp38-win_amd64.pyd
   from C:\Users\raman\AppData\Local\Temp\pip-uninstal

In [6]:
import numpy as np
import os
import scipy
from scipy.misc import imread, imresize
import datetime
import os

ImportError: cannot import name 'imread' from 'scipy.misc' (C:\Users\raman\anaconda3\lib\site-packages\scipy\misc\__init__.py)

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/notebooks/storage/Final_data/Collated_training/train'
val_path = '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = # choose the number of epochs
print ('# epochs =', num_epochs)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser = #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)